# Eye movement analysis, focus on Areas of Interest (AOI)
In this notebook the dataset located in **path** (see below), was merged with the AOI from the text file located in **file_name**. The result is a dataframe wich can be easily plotted and explored and compared by participant name, age, AOI, type of media (Abstract, Mixed, real, etc.). At the end of the notebook there are some initial plots made with seaborn summariying the results. In this Notebook we presented the results from **path 1**.

In [1]:
import re
import os
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from PIL import Image, ImageChops
import math, pywt
import seaborn as sns

# Path for the raw data, obtain file list

In [2]:
path = "D:\\MMData"
files = os.listdir(path)

# Importing the Textfile and extracting the order of the AOIs

In [3]:
file_name = "C:\\Users\\Jeremy_Constantin\\Documents\\Visdom plus plus\\P1M_All_AOIs_3Part1_DE.txt"
file_name_P2 = "C:\\Users\\Jeremy_Constantin\\Documents\\Visdom plus plus\\P2M_All_AOIs.txt"

# Functions

**Extract the initial line number for each AIO**

In [4]:
def search_string_in_file(file_name, string_to_search):
    """Search for the given string in file and return lines containing that string,
    along with line numbers"""
    line_number = 0

    # Open the file in read only mode
    with open(file_name, 'r') as read_obj:
        # Read all lines in the file one by one
        for line in read_obj:
            # For each line, check if line contains the string
            line_number += 1
            if string_to_search in line:
                # If yes, then add the line number & line as a tuple in the list
                Start_AOI_serie = line_number
    # Return list of tuples containing line numbers and lines where string is found
    return Start_AOI_serie


**Obtain frame and vertix number**

In [5]:
def get_frame_and_vertix_number(file_name, End_serie_index):
    
    count=1
    file1 = open(file_name, 'r') 
    kfRegex = re.compile(r'Number of Keyframes: (\d)+')
    nvRegex = re.compile(r'Number of Vertices: (\d)+')


    # Here is only for section B1, but we can check if it works with all
    while count<=End_serie_index: 
        count += 1
  
        # Get next line from file 
        line = file1.readline() 
  
        # if line is empty 
        # end of file is reached 
        if not line: 
            break
    
        m = kfRegex.search(line)
        if m:
            #print(m.group())
            str = m.group()
            num = [int(s) for s in str.split() if s.isdigit()] 
            kf=num[0]
            
        
        m1 = nvRegex.search(line)
        if m1:
            #print(m.group())
            str = m1.group()
            num1 = [int(s) for s in str.split() if s.isdigit()] 
            v=num1[0]
           
    file1.close()
    return kf, v

**Obtain time stamps and check if active**

In [6]:
def get_ts_active(file_name, End_serie_index):

    file1 = open(file_name, 'r') 
    tsRegex = re.compile(r'Keyframe timestamp = \d\d:\d\d:\d\d.\d\d\d')
    actFalseRegex = re.compile(r'Active: False')
    actTrueRegex = re.compile(r'Active: True')
    count = 0
    flag = 0
    flag1 = 0
    start_count_ts=0 # In order to get the time-stamps for Comparison with Raw Data
    end_count_ts=0
    ts=[]
    linets=[]
    lineact=[]



    #while count<=end_count_B1: 
    while count<=End_serie_index: 
        count += 1
  
        # Get next line from file 
        line = file1.readline() 
  
        # if line is empty 
        # end of file is reached 
        if not line: 
            break
    
        m = tsRegex.search(line)
        if m:
            ts.append(m.group())
            linets.append(count)
            #start_count_ts=count
            flag=flag+1
    
        n = actFalseRegex.search(line)
        if n:
            ts.append(n.group())
            lineact.append(0)
            #start_count_ts=count
            flag1=flag1+1
    
        o = actTrueRegex.search(line)
        if o:
            ts.append(o.group())
            lineact.append(1)
            #start_count_ts=count
            flag1=flag1+1
    return linets, lineact
        

# the define_AOI_series may not work

**Obtaining the coordinates**

In [7]:

def obtain_coordinates(file_name, Start_serie_ts, Vertix_number):
    file1 = open(file_name, 'r')
    count = 0
    flag = 0
    # Now we can use the ts in the data frame
    #start_count_c=linets[0] # In order to get the time-stamps for Comparison with Raw Data
    start_count_c= Start_serie_ts
    v = Vertix_number

    pts = []


    while flag==0: 
        count += 1
        line = file1.readline()
    
        if not line: 
            break
# Had to replace v, it was 11 for v-2 = 9   and then works but not so sure it is what we want...  
# Now is again fine. it was wrong before...
        if count>=(start_count_c+5) and count<(start_count_c+5+v):    
#    if count>=(start_count_c+5) and count<(start_count_c+5+v):
            row = line.split()
            for i in range(len(row)):
                row[i] = float(row[i])
            pts.append(row)
        
        if count>=(start_count_c+5+v):
            flag=1
    
    return pts


**Use to obtain in raw data the AOI series**

In [8]:
def define_AOI_series(x):
    y = None
    count = 0
    for i in range(len(df_s['ts_list'])):

        if (x >= df_s['ts_list'].loc[i][0]) &  (x <= df_s['ts_list'].loc[i][len(df_s['ts_list'].loc[i])-1]) :
            y = df_s['AOI_series'].loc[i]
        else:
            None
    count = count + 1
#    print(count, end="")
        
    return y

**Find AOI coordinates**

In [9]:
def AOI_coordinates (time, df_AOI_serie):

    if time == 0:
        coordinates = df_AOI_serie['Coodinates'][0]
        Active = df_AOI_serie['Active'][0]
    elif time > df_AOI_serie['Time_stamp_ms'][len(df_AOI_serie)-1]:
        coordinates = 'NaN'
        Active = 0
    else:
        df_AOI_reduced = df_AOI_serie[(df_AOI_serie['Time_stamp_ms']<=(time))&(df_AOI_serie['Time_stamp_ms']>=(time-300))].reset_index()
        if len(df_AOI_reduced['Time_stamp_ms']) <= 0:
            df_AOI_reduced = df_AOI_serie[(df_AOI_serie['Time_stamp_ms']<=(time))&(df_AOI_serie['Time_stamp_ms']>=(time-500))].reset_index()
            if len(df_AOI_reduced['Time_stamp_ms']) <= 0:
                df_AOI_reduced = df_AOI_serie[(df_AOI_serie['Time_stamp_ms']<=(time))&(df_AOI_serie['Time_stamp_ms']>=(time-1000))].reset_index()
                if len(df_AOI_reduced['Time_stamp_ms']) <= 0:
#                    print('Got nothing')
                    coordinates = 'NaN'
                    Active = 0
                else:
                    coordinates = df_AOI_reduced['Coodinates'][len(df_AOI_reduced)-1] 
                    Active = df_AOI_reduced['Active'][len(df_AOI_reduced)-1]
                
            else:
                coordinates = df_AOI_reduced['Coodinates'][len(df_AOI_reduced)-1]
                Active = df_AOI_reduced['Active'][len(df_AOI_reduced)-1]
        else:
            coordinates = df_AOI_reduced['Coodinates'][len(df_AOI_reduced)-1]
            Active = df_AOI_reduced['Active'][len(df_AOI_reduced)-1]
                    
    return coordinates, Active

**Is point inside a polygon** 


In [10]:
def point_inside_polygon(x, y, poly, include_edges=True):
    '''
    Test if point (x,y) is inside polygon poly.

    poly is N-vertices polygon defined as 
    [(x1,y1),...,(xN,yN)] or [(x1,y1),...,(xN,yN),(x1,y1)]
    (function works fine in both cases)

    Geometrical idea: point is inside polygon if horizontal beam
    to the right from point crosses polygon even number of times. 
    Works fine for non-convex polygons.
    '''
    if poly != 'NaN':
        n = len(poly)
        inside = False

        p1x, p1y = poly[0]
        for i in range(1, n + 1):
            p2x, p2y = poly[i % n]
            if p1y == p2y:
                if y == p1y:
                    if min(p1x, p2x) <= x <= max(p1x, p2x):
                        # point is on horisontal edge
                        inside = include_edges
                        break
                    elif x < min(p1x, p2x):  # point is to the left from current edge
                        inside = not inside
            else:  # p1y!= p2y
                if min(p1y, p2y) <= y <= max(p1y, p2y):
                    xinters = (y - p1y) * (p2x - p1x) / float(p2y - p1y) + p1x

                    if x == xinters:  # point is right on the edge
                        inside = include_edges
                        break

                    if x < xinters:  # point is to the left from current edge
                        inside = not inside

            p1x, p1y = p2x, p2y
    else:
        inside = 'NaN'

    return inside

**Get ploting names** Function is used for ploting labels

In [11]:
def exp_section_name(x):
    if (x == 'P1A.avi'):
        exp_Type = "Abstract"        
    elif (x == 'P2A.avi'):
        exp_Type = "Abstract"
    elif (x == 'P2M.avi'):
        exp_Type = "Mixed"        
    elif(x == 'P1M.avi'):
        exp_Type = "Mixed"
    elif (x == 'P1R.avi'):
        exp_Type = "Real"
    elif (x == 'P2R.avi'):
        exp_Type = "Real"
    else:
        exp_Type = 'NaN'
    return exp_Type



**Get_all_data** allow to read all the files in one go as well as extract all the information about the start of diferent parts in the experiment. Returns a df

In [12]:
def get_all_data(path, selected_variables):
   
    files = os.listdir(path)
    # little silly but we need  
    df = pd.DataFrame()
    
    for f, file in enumerate(files):
        print("Loading:", f+1, " / ", len(files), " files. File name: ",  file)
        # Load the dataframe for each file
                
        dt = pd.read_csv(path + "/" + file, sep='\t')
        dt = dt[selected_variables]
        
        # Initialize the dataframe which will be retuned   
#        df = pd.DataFrame(columns = dt.columns)
    
        # Get the sections start and end time for each aprticipant    
        uniqueValuest, indicesListt, countListt = np.unique(list(dt.MediaName), return_index=True, return_counts=True)
        #
        df1 = pd.DataFrame(columns = ['ParticipantName','StudioTestName', 'MediaName', 'Start_Section']) 
        df1['MediaName']=pd.Series(uniqueValuest)
#        print(df1['Exp_Section'])
        df1['Start_Section'] = pd.Series(indicesListt)
        df1['countListt'] = pd.Series(countListt)
        df1['StudioTestName'] = dt.StudioTestName[0]
        df1['ParticipantName'] = dt.ParticipantName[0]
       
        # Sort the values in cronological order
        df1.sort_values(by=['Start_Section'], inplace=True, ignore_index=True)
        
        # Obtain the timestamps of the starting sections
        df1['Start_Timestamp'] = pd.Series(list(dt['RecordingTimestamp'][df1['Start_Section'][:]]))
        
        # drop colums that are not needed
        df1.drop(columns=['Start_Section', 'countListt'], inplace=True)
        # Merge data frames
        df1 = df1.merge(dt, on=['MediaName', 'ParticipantName', 'StudioTestName'])
        df1['Timestamp_n'] = df1['RecordingTimestamp'] - df1['Start_Timestamp'] 
        # Append each participant to make a df containing all participants
#\df1['AOI_Section_Type'] = df1[df1['MediaName'] == ('P1A.avi' or 'P2A.avi' or 'P2M.avi'or'P1M.avi'or 'P1R.avi' or 'P2R.avi')]['Timestamp_n'].transform(define_AOI_series)
#        df1['AOI_Section_Type'] = df1[df1['MediaName'] == ('P1A.avi' or 'P2A.avi' or 'P2M.avi'or'P1M.avi'or 'P1R.avi' or 'P2R.avi')]['Timestamp_n'].transform(define_AOI_series)



#        df1['AOI_Section_Type'] = df1[(df1['MediaName'] == 'P1A.avi') | (df1['MediaName'] =='P1A.avi') | (df1['MediaName'] =='P2A.avi') | 
#                                      (df1['MediaName'] =='P2M.avi')| (df1['MediaName'] =='P1M.avi')| (df1['MediaName'] =='P1R.avi') | 
#                                      (df1['MediaName'] =='P2R.avi')]['Timestamp_n'].transform(define_AOI_series)
#    
#        df1['Coordinates'] = df1[(df1['MediaName'] == 'P1A.avi') | (df1['MediaName'] =='P1A.avi') | (df1['MediaName'] =='P2A.avi') | 
#                                    (df1['MediaName'] =='P2M.avi')| (df1['MediaName'] =='P1M.avi')| (df1['MediaName'] =='P1R.avi') | 
#                                    (df1['MediaName'] =='P2R.avi')]['Timestamp_n'].transform(AOI_coordinates)
        df = df.append(df1) 
    # Getting sections names    
    df['Section_Type'] = df['MediaName'].transform(exp_section_name)
    # Splitong by age
    df["Age"] = df["ParticipantName"].str.strip('0123456789.A')
    

 
    return df
    

In [13]:
def get_all_data_Path1(path, selected_variables, AOI_serie):
   
    files = os.listdir(path)
    # little silly but we need  
    df = pd.DataFrame()
    df1_encoding_Path1 = pd.DataFrame()
    for f, file in enumerate(files):
        print("Loading:", f+1, " / ", len(files), " files. File name: ",  file)
        # Load the dataframe for each file
                
        dt = pd.read_csv(path + "/" + file, sep='\t')
        dt = dt[selected_variables]
        
        # Initialize the dataframe which will be retuned   
#        df = pd.DataFrame(columns = dt.columns)
    
        # Get the sections start and end time for each aprticipant    
        uniqueValuest, indicesListt, countListt = np.unique(list(dt.MediaName), return_index=True, return_counts=True)
        #
        df1 = pd.DataFrame(columns = ['ParticipantName','StudioTestName', 'MediaName', 'Start_Section']) 
        df1['MediaName']=pd.Series(uniqueValuest)
#        print(df1['Exp_Section'])
        df1['Start_Section'] = pd.Series(indicesListt)
        df1['countListt'] = pd.Series(countListt)
        df1['StudioTestName'] = dt.StudioTestName[0]
        df1['ParticipantName'] = dt.ParticipantName[0]
       
        # Sort the values in cronological order
        df1.sort_values(by=['Start_Section'], inplace=True, ignore_index=True)
        
        # Obtain the timestamps of the starting sections
        df1['Start_Timestamp'] = pd.Series(list(dt['RecordingTimestamp'][df1['Start_Section'][:]]))
        
        # drop colums that are not needed
        df1.drop(columns=['Start_Section', 'countListt'], inplace=True)
        # Merge data frames
        df1 = df1.merge(dt, on=['MediaName', 'ParticipantName', 'StudioTestName'])
        df1['Timestamp_n'] = df1['RecordingTimestamp'] - df1['Start_Timestamp']
        #
        
        # For now only path one
        df1_encoding_Path11 = df1[(df1['MediaName'] == 'P1A.avi') | (df1['MediaName'] =='P1M.avi')| (df1['MediaName'] =='P1R.avi')].reset_index()

        for i, AOI_serie in enumerate(AOI_series):
            df_AOI_serie = df_AOI[df_AOI['AOI_series']==AOI_serie].reset_index()
            coord_list_t =[]
            active_list_t = []
            print('Making: ', AOI_serie )
            for i, time in enumerate(df1_encoding_Path11['Timestamp_n']):
                coord, act =  AOI_coordinates(time, df_AOI_serie)
                coord_list_t.append(coord)  
                active_list_t.append(act)
            df1_encoding_Path11['Coord_'+AOI_serie] = pd.Series(coord_list_t)
            df1_encoding_Path11['Active_'+AOI_serie] = pd.Series(active_list_t)
            
            
            
        x_param = 'StrictAverageGazePointX (ADCSmm)'
        y_param ='StrictAverageGazePointY (ADCSmm)'

        z_param = 'FixationPointX (MCSpx)'
        v_param = 'FixationPointY (MCSpx)'

        for i, AOI_serie in enumerate(AOI_series):
            is_in_list_Gaze = []
            is_in_list_Fix = []
            for i, poly in enumerate(df1_encoding_Path11['Coord_' + AOI_serie]):
                x = df1_encoding_Path11[x_param ][i]
                y = df1_encoding_Path11[y_param][i]
                z = df1_encoding_Path11[z_param ][i]
                v = df1_encoding_Path11[v_param][i]
        
                poly = df1_encoding_Path11['Coord_' + AOI_serie][i]
#    print(i, poly)
                is_in_Gaze = point_inside_polygon(x, y, poly, include_edges=True)
                is_in_Fix = point_inside_polygon(z, v, poly, include_edges=True)
                if is_in_Gaze == True:
                    in_Gaze = 1
                else:
                    in_Gaze = 0
                if is_in_Fix == True:
                    in_Fix = 1
                else:
                    in_Fix = 0
                    
                is_in_list_Gaze.append(in_Gaze)
                is_in_list_Fix.append(in_Fix)
        
    
            df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Avg_Gaze'] = pd.Series(is_in_list_Gaze)
            df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Fix_Point'] = pd.Series(is_in_list_Fix) 

        df = df.append(df1)
        df1_encoding_Path1 = df1_encoding_Path1.append(df1_encoding_Path11)
    # Getting sections names    
    df['Section_Type'] = df['MediaName'].transform(exp_section_name)
    # Splitong by age
    df["Age"] = df["ParticipantName"].str.strip('0123456789.A')
    

 
    return df, df1_encoding_Path1   
    


# AIOs

**Obtain AIO names**

In [14]:
import re

file1 = open(file_name, 'r') 
count = 0

B_series=[]
T_series=[]
Build_series=[]
Route_series=[]
AOI_series=[]

  
while True: 
  
    # Get next line from file 
    line = file1.readline() 
  
    # if line is empty 
    # end of file is reached 
    if not line: 
        break
    m = re.match("AOI name: (B[0-9])", line)
    m1 = re.match("AOI name: (T[0-9])", line)
    m2 = re.match("AOI name: (Build[0-9])", line)
    m3 = re.match("AOI name: (Route[0-9]*)", line)
    
    
    
    if m:
        B_series.append(m.group(1))
        AOI_series.append(m.group(1))
    
    if m1:
        T_series.append(m1.group(1))
        AOI_series.append(m1.group(1))
        
    if m2:
        Build_series.append(m2.group(1))
        AOI_series.append(m2.group(1))
        
    if m3:
        Route_series.append(m3.group(1))
        AOI_series.append(m3.group(1))
    

print("B_series=",B_series)
print("T_series=",T_series)
print("Build_series=",Build_series)
print("Route_series=",Route_series)
print("AOI_series order=",AOI_series)

file1.close() 

B_series= ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
T_series= ['T1', 'T2', 'T3', 'T4', 'T5']
Build_series= ['Build1', 'Build2', 'Build3', 'Build4']
Route_series= ['Route']
AOI_series order= ['B1', 'Build1', 'T1', 'Build2', 'Build3', 'Route', 'B2', 'Build4', 'B3', 'B4', 'T2', 'B5', 'B6', 'T3', 'T4', 'B7', 'T5']


# Extracting the data, maybe here change file_name this is for Path 1

**Obtain Start and End series_index list** 

In [15]:
list_Start_serie_index =[]
list_End_serie_index =[]
for y, val in enumerate(AOI_series):
    string_to_search = val
#    Start_serie_index = search_string_in_file('P1M_All_AOIs_3Part1_DE.txt', string_to_search)
    Start_serie_index = search_string_in_file(file_name, string_to_search)
    list_Start_serie_index.append(Start_serie_index)


In [16]:
# Obtain the number of the last line in the file
with open(file_name, 'r') as fp:
    Final_serie_index = len(fp.readlines())

    
for i, val in enumerate(AOI_series):
    if i < (len(AOI_series)-1):
        End_serie_index = list_Start_serie_index[i+1]-1
        list_End_serie_index.append(End_serie_index)
    else:
        End_serie_index = Final_serie_index
        list_End_serie_index.append(End_serie_index)

**Keyframe and Vertices Detection** 

In [17]:
keyframes_number_list = []
Vertix_number_list  = []
for y, val in enumerate(list_End_serie_index):
    kf, v = get_frame_and_vertix_number(file_name, val)

    keyframes_number_list.append(kf)
    Vertix_number_list.append(v)

# Make a DataFrame with the series information

In [18]:
df_s = pd.DataFrame(columns = ['AOI_series','Start_serie_index', 'End_serie_index']) 
df_s['AOI_series'] = pd.Series(AOI_series)
df_s['Start_serie_index'] = pd.Series(list_Start_serie_index)
df_s['End_serie_index'] = pd.Series(list_End_serie_index)
df_s['keyframes_number'] = pd.Series(keyframes_number_list)
df_s['keyframes_cum'] = df_s['keyframes_number'].cumsum()
df_s['Vertix_number'] = pd.Series(Vertix_number_list)
df_s

,AOI_series,Start_serie_index,End_serie_index,keyframes_number,keyframes_cum,Vertix_number
0,B1,1,336,22,22,9
1,Build1,337,5802,390,412,8
2,T1,5803,6228,30,442,8
3,Build2,6229,12879,443,885,9
4,Build3,12880,17813,308,1193,10
5,Route,17814,25707,464,1657,11
6,B2,25708,26581,62,1719,8
7,Build4,26582,30157,255,1974,8
8,B3,30158,31621,81,2055,12
9,B4,31622,32131,36,2091,8


**Obtain time stamps and active frames**

In [19]:
ts_full, act_full = get_ts_active(file_name, list_End_serie_index[16])

In [20]:
ts_list = []
act_list  = []


for y, val in enumerate(df_s['keyframes_cum']):
    if y == 0:
        ts = ts_full[0:(val)]
        act = act_full[0:(val)]
    else:
        ts= ts_full[df_s['keyframes_cum'][y-1]:(val)]
        act= act_full[df_s['keyframes_cum'][y-1]:(val)]
    ts_list.append(ts)
    act_list .append(act)

**Include the time stamps and active zones in the data frame**

In [21]:
df_s['ts_list'] = pd.Series(ts_list)
df_s['act_list'] = pd.Series(act_list)
df_s

,AOI_series,Start_serie_index,End_serie_index,keyframes_number,keyframes_cum,Vertix_number,ts_list,act_list
0,B1,1,336,22,22,9,"[6, 21, 36, 51, 66, 81, 96, 111, 126, 141, 156...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
1,Build1,337,5802,390,412,8,"[342, 356, 370, 384, 398, 412, 426, 440, 454, ...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,T1,5803,6228,30,442,8,"[5808, 5822, 5836, 5850, 5864, 5878, 5892, 590...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,Build2,6229,12879,443,885,9,"[6234, 6249, 6264, 6279, 6294, 6309, 6324, 633...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,Build3,12880,17813,308,1193,10,"[12885, 12901, 12917, 12933, 12949, 12965, 129...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
5,Route,17814,25707,464,1657,11,"[17819, 17836, 17853, 17870, 17887, 17904, 179...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
6,B2,25708,26581,62,1719,8,"[25713, 25727, 25741, 25755, 25769, 25783, 257...","[0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
7,Build4,26582,30157,255,1974,8,"[26587, 26601, 26615, 26629, 26643, 26657, 266...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
8,B3,30158,31621,81,2055,12,"[30163, 30181, 30199, 30217, 30235, 30253, 302...","[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
9,B4,31622,32131,36,2091,8,"[31627, 31641, 31655, 31669, 31683, 31697, 317...","[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


**Obtain the coordinates**

In [22]:
coordinates_list = []
coordinates_list_full = []
AOI_serie_full = []
for y, val in enumerate(df_s['Vertix_number']):
    coordinates_list_t = []
    for z, time in enumerate(df_s['ts_list'].loc[y]):
        Start_serie_ts = df_s['ts_list'][y][z]
        Vertix_number = df_s['Vertix_number'][y]
        pts = obtain_coordinates(file_name, Start_serie_ts, Vertix_number)
        coordinates_list_full.append(pts)
        AOI_serie_full.append(df_s['AOI_series'].loc[y])
        coordinates_list_t.append(pts)
    coordinates_list.append(coordinates_list_t)

# Making df_AOI
**Just extracting relevant values**

In [23]:
AOI_serie_full = []

for i in range(len(df_s)):
#    print(df_s['ts_list'].loc[i][0], df_s['ts_list'].loc[i][len(df_s['ts_list'].loc[i])-1])
    for y, val in enumerate(df_s['ts_list'].loc[i]):
        AOI_serie_full.append(df_s['AOI_series'].loc[i])
#        print(val, df_s['AOI_series'].loc[i] )
   

In [24]:
#len(coordinates_list_full), len(ts_full), len(act_full), len(AOI_serie_full)
df_AOI = pd.DataFrame(list(zip(AOI_serie_full, ts_full, act_full, coordinates_list_full)), columns = ['AOI_series','Time_stamp_line', 'Active', 'Coodinates']) 

df_AOI

,AOI_series,Time_stamp_line,Active,Coodinates
0,B1,6,0,"[[1215.02, 6.7], [1218.37, 335.18], [1221.72, ..."
1,B1,21,1,"[[1215.02, 6.7], [1218.37, 335.18], [1221.72, ..."
2,B1,36,1,"[[1260.06, -2.87], [1264.53, 341.73], [1256.88..."
3,B1,51,1,"[[1280.51, -2.08], [1280.02, 343.93], [1268.67..."
4,B1,66,1,"[[1349.04, 0.79], [1348.53, 360.01], [1350.1, ..."
...,...,...,...,...
2565,T5,38781,1,"[[938.5, 183.92], [773.02, -4.04], [160.18, 1...."
2566,T5,38795,1,"[[938.5, 183.43], [793.82, -0.69], [108.23, 1...."
2567,T5,38809,1,"[[938.5, 185.84], [787.66, -4.04], [-4.05, 1.9..."
2568,T5,38823,1,"[[918.38, 155.67], [816.15, -0.69], [-4.05, 1...."


In [25]:
# Still something not rigth here
# This is fine then we need to match one by one....

In [26]:
file1 = open(file_name, "r")
lines = file1.readlines()
time_ms = []
for i in range(len(df_AOI['Time_stamp_line'])):
    time_strip = lines[df_AOI['Time_stamp_line'][i]-1].strip('Keyframe timestamp = \n')
    time = pd.to_datetime("1970-01-01 "+time_strip, format='%Y-%m-%d %H:%M:%S.%f')
    time = time.timestamp()*1000
    time_ms.append(int(time))
#    time = pd.to_datetime("1970-01-01"+lines[df_AOI['Time_stamp'][i]])
# Add the time?stamp in ms to the df
df_AOI['Time_stamp_ms'] = pd.Series(time_ms)


In [27]:
df_AOI

,AOI_series,Time_stamp_line,Active,Coodinates,Time_stamp_ms
0,B1,6,0,"[[1215.02, 6.7], [1218.37, 335.18], [1221.72, ...",0
1,B1,21,1,"[[1215.02, 6.7], [1218.37, 335.18], [1221.72, ...",333
2,B1,36,1,"[[1260.06, -2.87], [1264.53, 341.73], [1256.88...",1426
3,B1,51,1,"[[1280.51, -2.08], [1280.02, 343.93], [1268.67...",1793
4,B1,66,1,"[[1349.04, 0.79], [1348.53, 360.01], [1350.1, ...",2856
...,...,...,...,...,...
2565,T5,38781,1,"[[938.5, 183.92], [773.02, -4.04], [160.18, 1....",88001
2566,T5,38795,1,"[[938.5, 183.43], [793.82, -0.69], [108.23, 1....",88168
2567,T5,38809,1,"[[938.5, 185.84], [787.66, -4.04], [-4.05, 1.9...",88293
2568,T5,38823,1,"[[918.38, 155.67], [816.15, -0.69], [-4.05, 1....",89294


# Making a test with only one participant

**Loading the files**

In [28]:
    files = os.listdir(path)
    # little silly but we need  
    df = pd.DataFrame()

**Choosing the variables**

In [29]:
selected_variables = ['StudioProjectName', 'StudioTestName',
       'ParticipantName', 'RecordingName', 'RecordingDate', 'MediaName', 'RecordingTimestamp', 'FixationIndex',
       'SaccadeIndex', 'GazeEventType', 'GazeEventDuration',
       'FixationPointX (MCSpx)', 'FixationPointY (MCSpx)', 'SaccadicAmplitude',
       'AbsoluteSaccadicDirection', 'RelativeSaccadicDirection', 'StrictAverageGazePointX (ADCSmm)',
       'StrictAverageGazePointY (ADCSmm)', 'PupilLeft', 'PupilRight']

**Obtain df1 for participant 0**

In [30]:
        dt = pd.read_csv(path + "/" + files[0], sep='\t')
        dt = dt[selected_variables]
        
        # Initialize the dataframe which will be retuned   
#        df = pd.DataFrame(columns = dt.columns)
    
        # Get the sections start and end time for each aprticipant    
        uniqueValuest, indicesListt, countListt = np.unique(list(dt.MediaName), return_index=True, return_counts=True)
        #
        df1 = pd.DataFrame(columns = ['ParticipantName','StudioTestName', 'MediaName', 'Start_Section']) 
        df1['MediaName']=pd.Series(uniqueValuest)
#        print(df1['Exp_Section'])
        df1['Start_Section'] = pd.Series(indicesListt)
        df1['countListt'] = pd.Series(countListt)
        df1['StudioTestName'] = dt.StudioTestName[0]
        df1['ParticipantName'] = dt.ParticipantName[0]
       
        # Sort the values in cronological order
        df1.sort_values(by=['Start_Section'], inplace=True, ignore_index=True)
        
        # Obtain the timestamps of the starting sections
        df1['Start_Timestamp'] = pd.Series(list(dt['RecordingTimestamp'][df1['Start_Section'][:]]))
        
        # drop colums that are not needed
        df1.drop(columns=['Start_Section', 'countListt'], inplace=True)
        # Merge data frames
        df1 = df1.merge(dt, on=['MediaName', 'ParticipantName', 'StudioTestName'])
        df1['Timestamp_n'] = df1['RecordingTimestamp'] - df1['Start_Timestamp']
        #


C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\3966427089.py:1: DtypeWarning: Columns (28,34,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv(path + "/" + files[0], sep='\t')


In [31]:
df1

,ParticipantName,StudioTestName,MediaName,Start_Timestamp,StudioProjectName,RecordingName,RecordingDate,RecordingTimestamp,FixationIndex,SaccadeIndex,...,FixationPointX (MCSpx),FixationPointY (MCSpx),SaccadicAmplitude,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n
0,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27246,94.0,NaN,...,776.0,599.0,6.99,174.29,174.99,NaN,NaN,NaN,NaN,0
1,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27248,94.0,NaN,...,776.0,599.0,6.99,174.29,174.99,862.67,582.10,5.14,5.28,2
2,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27251,94.0,NaN,...,776.0,599.0,6.99,174.29,174.99,862.57,561.98,5.12,5.27,5
3,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27255,94.0,NaN,...,776.0,599.0,6.99,174.29,174.99,860.52,555.06,5.16,5.29,9
4,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27258,94.0,NaN,...,776.0,599.0,6.99,174.29,174.99,857.68,559.16,5.12,5.29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452368,o02.01.A,01-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,1593586,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1593982,3280.0,NaN,...,287.0,232.0,4.06,174.74,33.13,542.20,905.95,4.70,4.86,396
452369,o02.01.A,01-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,1593586,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1593986,3280.0,NaN,...,287.0,232.0,4.06,174.74,33.13,537.73,904.89,4.72,4.86,400
452370,o02.01.A,01-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,1593586,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1593989,3280.0,NaN,...,287.0,232.0,4.06,174.74,33.13,562.29,899.67,4.70,4.89,403
452371,o02.01.A,01-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,1593586,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1593993,3280.0,NaN,...,287.0,232.0,4.06,174.74,33.13,563.16,891.46,4.66,4.85,407


In [32]:
tempo = df1[df1['PupilLeft'].notna()]
left_avg_pupil_dill = tempo['PupilLeft'].astype('float').diff().mean()
tempo = tempo['RecordingTimestamp'].diff()

blink_counter = sum( (tempo >= 25) & (tempo < 45) )

blink_counter
left_avg_pupil_dill

-3.041112033300171e-06

**Select only the videos for path 1**

In [33]:

        df1_encoding_Path11 = df1[(df1['MediaName'] == 'P1A.avi') | (df1['MediaName'] =='P1M.avi')| (df1['MediaName'] =='P1R.avi')].reset_index()
        df1_encoding_Path11
        
        

,index,ParticipantName,StudioTestName,MediaName,Start_Timestamp,StudioProjectName,RecordingName,RecordingDate,RecordingTimestamp,FixationIndex,...,FixationPointX (MCSpx),FixationPointY (MCSpx),SaccadicAmplitude,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n
0,0,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27246,94.0,...,776.0,599.0,6.99,174.29,174.99,NaN,NaN,NaN,NaN,0
1,1,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27248,94.0,...,776.0,599.0,6.99,174.29,174.99,862.67,582.10,5.14,5.28,2
2,2,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27251,94.0,...,776.0,599.0,6.99,174.29,174.99,862.57,561.98,5.12,5.27,5
3,3,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27255,94.0,...,776.0,599.0,6.99,174.29,174.99,860.52,555.06,5.16,5.29,9
4,4,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27258,94.0,...,776.0,599.0,6.99,174.29,174.99,857.68,559.16,5.12,5.29,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53813,326180,o02.01.A,01-Part1-DE,P1R.avi,1076151,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1165720,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89569
53814,326181,o02.01.A,01-Part1-DE,P1R.avi,1076151,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1165723,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89572
53815,326182,o02.01.A,01-Part1-DE,P1R.avi,1076151,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1165727,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89576
53816,326183,o02.01.A,01-Part1-DE,P1R.avi,1076151,IsminiLokkaExperiment1,o02.01.A,11/3/2016,1165730,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89579


**Below we have changed the function which is now working, have been alreadz replaced up**

In [34]:
        for i, AOI_serie in enumerate(AOI_series):
            df_AOI_serie = df_AOI[df_AOI['AOI_series']==AOI_serie].reset_index()
            coord_list_t =[]
            active_list_t = []
            print('Making: ', AOI_serie )
            for i, time in enumerate(df1_encoding_Path11['Timestamp_n']):
                coord, act =  AOI_coordinates(time, df_AOI_serie)
                coord_list_t.append(coord)  
                active_list_t.append(act)
            df1_encoding_Path11['Coord_'+AOI_serie] = pd.Series(coord_list_t)
            df1_encoding_Path11['Active_'+AOI_serie] = pd.Series(active_list_t)


Making:  B1
Making:  Build1
Making:  T1
Making:  Build2
Making:  Build3
Making:  Route
Making:  B2
Making:  Build4
Making:  B3
Making:  B4
Making:  T2
Making:  B5
Making:  B6
Making:  T3
Making:  T4
Making:  B7
Making:  T5


In [35]:
AOI_serie = AOI_series[16]
print(AOI_serie)
df1_encoding_Path11['Coord_'+AOI_serie]


T5


0        [[117.31, 273.17], [62.01, 229.6], [0.0, 226.2...
1        [[117.31, 273.17], [62.01, 229.6], [0.0, 226.2...
2        [[117.31, 273.17], [62.01, 229.6], [0.0, 226.2...
3        [[117.31, 273.17], [62.01, 229.6], [0.0, 226.2...
4        [[117.31, 273.17], [62.01, 229.6], [0.0, 226.2...
                               ...                        
53813    [[918.38, 155.67], [816.15, -0.69], [-4.05, 1....
53814    [[918.38, 155.67], [816.15, -0.69], [-4.05, 1....
53815    [[918.38, 155.67], [816.15, -0.69], [-4.05, 1....
53816    [[918.38, 155.67], [816.15, -0.69], [-4.05, 1....
53817    [[918.38, 155.67], [816.15, -0.69], [-4.05, 1....
Name: Coord_T5, Length: 53818, dtype: object

In [36]:
df1_encoding_Path11['Active_'+AOI_serie].value_counts()

0    47779
1     6039
Name: Active_T5, dtype: int64

In [37]:
        
        

        x_param = 'StrictAverageGazePointX (ADCSmm)'
        y_param ='StrictAverageGazePointY (ADCSmm)'

        z_param = 'FixationPointX (MCSpx)'
        v_param = 'FixationPointY (MCSpx)'

        for i, AOI_serie in enumerate(AOI_series):
            is_in_list_Gaze = []
            is_in_list_Fix = []
            for i, poly in enumerate(df1_encoding_Path11['Coord_' + AOI_serie]):
                x = df1_encoding_Path11[x_param ][i]
                y = df1_encoding_Path11[y_param][i]
                z = df1_encoding_Path11[z_param ][i]
                v = df1_encoding_Path11[v_param][i]
        
                poly = df1_encoding_Path11['Coord_' + AOI_serie][i]
#    print(i, poly)
                is_in_Gaze = point_inside_polygon(x, y, poly, include_edges=True)
                is_in_Fix = point_inside_polygon(z, v, poly, include_edges=True)
                if is_in_Gaze == True:
                    in_Gaze = 1
                else:
                    in_Gaze = 0
                if is_in_Fix == True:
                    in_Fix = 1
                else:
                    in_Fix = 0
                    
                is_in_list_Gaze.append(in_Gaze)
                is_in_list_Fix.append(in_Fix)
        
    
            df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Avg_Gaze'] = pd.Series(is_in_list_Gaze)
            df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Fix_Point'] = pd.Series(is_in_list_Fix) 

In [38]:
AOI_serie = AOI_series[16]
df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Avg_Gaze'].value_counts()

0    53799
1       19
Name: Inside_T5_AIO_Avg_Gaze, dtype: int64

# Data Collections + Blink counter + average pupil Change 

In [39]:
def get_all_data_Path1(path, selected_variables, AOI_serie):
   
    files = os.listdir(path)
    # little silly but we need  
    df = pd.DataFrame()
    df1_encoding_Path1 = pd.DataFrame()
    for f, file in enumerate(files[0:150]):
        print("Loading:", f+1, " / ", len(files), " files. File name: ",  file)
        # Load the dataframe for each file
                
        dt = pd.read_csv(path + "/" + file, sep='\t')
        dt = dt[selected_variables]
        
        # Initialize the dataframe which will be retuned   
#        df = pd.DataFrame(columns = dt.columns)
    
        # Get the sections start and end time for each aprticipant    
        uniqueValuest, indicesListt, countListt = np.unique(list(dt.MediaName), return_index=True, return_counts=True)
        #
        df1 = pd.DataFrame(columns = ['ParticipantName','StudioTestName', 'MediaName', 'Start_Section']) 
        df1['MediaName']=pd.Series(uniqueValuest)
#        print(df1['Exp_Section'])
        df1['Start_Section'] = pd.Series(indicesListt)
        df1['countListt'] = pd.Series(countListt)
        df1['StudioTestName'] = dt.StudioTestName[0]
        df1['ParticipantName'] = dt.ParticipantName[0]
        
        #Blink counter start
        tempo = dt[dt['PupilLeft'].notna()]
        left_avg_pupil_dill = tempo['PupilLeft'].astype('float').diff().mean()
        tempo = tempo['RecordingTimestamp'].diff()
        blink_counter = sum( (tempo >= 25) & (tempo < 45) )
        df1['Blink_Counter'] = blink_counter
        df1['left_avg_pupil_dill'] = left_avg_pupil_dill
        #Blink counter end
           
        # Sort the values in cronological order
        df1.sort_values(by=['Start_Section'], inplace=True, ignore_index=True)
        
        # Obtain the timestamps of the starting sections
        df1['Start_Timestamp'] = pd.Series(list(dt['RecordingTimestamp'][df1['Start_Section'][:]]))
        
        # drop colums that are not needed
        df1.drop(columns=['Start_Section', 'countListt'], inplace=True)
        # Merge data frames
        df1 = df1.merge(dt, on=['MediaName', 'ParticipantName', 'StudioTestName'])
        df1['Timestamp_n'] = df1['RecordingTimestamp'] - df1['Start_Timestamp']
        
        #
        
        # For now only path one
        df1_encoding_Path11 = df1[(df1['MediaName'] == 'P1A.avi') | (df1['MediaName'] =='P1M.avi')| (df1['MediaName'] =='P1R.avi')].reset_index()

        for i, AOI_serie in enumerate(AOI_series):
            df_AOI_serie = df_AOI[df_AOI['AOI_series']==AOI_serie].reset_index()
            coord_list_t =[]
            active_list_t = []
            print('Making: ', AOI_serie )
            for i, time in enumerate(df1_encoding_Path11['Timestamp_n']):
                coord, act =  AOI_coordinates(time, df_AOI_serie)
                coord_list_t.append(coord)  
                active_list_t.append(act)
            df1_encoding_Path11['Coord_'+AOI_serie] = pd.Series(coord_list_t)
            df1_encoding_Path11['Active_'+AOI_serie] = pd.Series(active_list_t)
            
            
            
        x_param = 'StrictAverageGazePointX (ADCSmm)'
        y_param ='StrictAverageGazePointY (ADCSmm)'

        z_param = 'FixationPointX (MCSpx)'
        v_param = 'FixationPointY (MCSpx)'

        for i, AOI_serie in enumerate(AOI_series):
            is_in_list_Gaze = []
            is_in_list_Fix = []
            for i, poly in enumerate(df1_encoding_Path11['Coord_' + AOI_serie]):
                x = df1_encoding_Path11[x_param ][i]
                y = df1_encoding_Path11[y_param][i]
                z = df1_encoding_Path11[z_param ][i]
                v = df1_encoding_Path11[v_param][i]
        
                poly = df1_encoding_Path11['Coord_' + AOI_serie][i]
#    print(i, poly)
                is_in_Gaze = point_inside_polygon(x, y, poly, include_edges=True)
                is_in_Fix = point_inside_polygon(z, v, poly, include_edges=True)
                if is_in_Gaze == True:
                    in_Gaze = 1
                else:
                    in_Gaze = 0
                if is_in_Fix == True:
                    in_Fix = 1
                else:
                    in_Fix = 0
                    
                is_in_list_Gaze.append(in_Gaze)
                is_in_list_Fix.append(in_Fix)
        
    
            df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Avg_Gaze'] = pd.Series(is_in_list_Gaze)
            df1_encoding_Path11['Inside_' + AOI_serie + '_AIO_Fix_Point'] = pd.Series(is_in_list_Fix) 

        df = df.append(df1)
        df1_encoding_Path1 = df1_encoding_Path1.append(df1_encoding_Path11)
    # Getting sections names    
    df['Section_Type'] = df['MediaName'].transform(exp_section_name)
    # Splitong by age
    df["Age"] = df["ParticipantName"].str.strip('0123456789.A')

    

 
    return df, df1_encoding_Path1
    

In [40]:
df, df1_encoding_Path1 =get_all_data_Path1(path, selected_variables, AOI_serie)

Loading: 1  /  172  files. File name:  IsminiLokkaExperiment1_01-Part1-DE_o02.01.A.tsv


C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:11: DtypeWarning: Columns (28,34,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv(path + "/" + file, sep='\t')


Making:  B1
Making:  Build1
Making:  T1
Making:  Build2
Making:  Build3
Making:  Route
Making:  B2
Making:  Build4
Making:  B3
Making:  B4
Making:  T2
Making:  B5
Making:  B6
Making:  T3
Making:  T4
Making:  B7
Making:  T5


C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)
C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_encoding_Path1 = df1_encoding_Path1.append(df1_encoding_Path11)


Loading: 2  /  172  files. File name:  IsminiLokkaExperiment1_01-Part1-DE_o31.01.A.tsv


...

C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)
C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_encoding_Path1 = df1_encoding_Path1.append(df1_encoding_Path11)


Loading: 150  /  172  files. File name:  IsminiLokkaExperiment1_06-Part1-DE_o30.06.A.tsv


C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:11: DtypeWarning: Columns (11,28,34,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv(path + "/" + file, sep='\t')


Making:  B1
Making:  Build1
Making:  T1
Making:  Build2
Making:  Build3
Making:  Route
Making:  B2
Making:  Build4
Making:  B3
Making:  B4
Making:  T2
Making:  B5
Making:  B6
Making:  T3
Making:  T4
Making:  B7
Making:  T5


C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:103: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df1)
C:\Users\Jeremy_Constantin\AppData\Local\Temp\ipykernel_53868\1969536936.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1_encoding_Path1 = df1_encoding_Path1.append(df1_encoding_Path11)


In [41]:
df

,ParticipantName,StudioTestName,MediaName,Blink_Counter,left_avg_pupil_dill,Start_Timestamp,StudioProjectName,RecordingName,RecordingDate,RecordingTimestamp,...,SaccadicAmplitude,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n,Section_Type,Age
0,o02.01.A,01-Part1-DE,P1A.avi,4106,-3.573414e-06,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27246,...,6.99,174.29,174.99,NaN,NaN,NaN,NaN,0,Abstract,o
1,o02.01.A,01-Part1-DE,P1A.avi,4106,-3.573414e-06,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27248,...,6.99,174.29,174.99,862.67,582.10,5.14,5.28,2,Abstract,o
2,o02.01.A,01-Part1-DE,P1A.avi,4106,-3.573414e-06,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27251,...,6.99,174.29,174.99,862.57,561.98,5.12,5.27,5,Abstract,o
3,o02.01.A,01-Part1-DE,P1A.avi,4106,-3.573414e-06,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27255,...,6.99,174.29,174.99,860.52,555.06,5.16,5.29,9,Abstract,o
4,o02.01.A,01-Part1-DE,P1A.avi,4106,-3.573414e-06,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27258,...,6.99,174.29,174.99,857.68,559.16,5.12,5.29,12,Abstract,o
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664288,o30.06.A,06-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,897,9.411233e-08,2302158,IsminiLokkaExperiment1,o30.06.A,11/30/2016,2302512,...,NaN,NaN,NaN,1130.82,557.19,2.99,2.90,354,NaN,o
664289,o30.06.A,06-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,897,9.411233e-08,2302158,IsminiLokkaExperiment1,o30.06.A,11/30/2016,2302515,...,NaN,NaN,NaN,1133.11,563.09,2.99,2.90,357,NaN,o
664290,o30.06.A,06-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,897,9.411233e-08,2302158,IsminiLokkaExperiment1,o30.06.A,11/30/2016,2302518,...,NaN,NaN,NaN,1134.02,562.97,2.99,2.90,360,NaN,o
664291,o30.06.A,06-Part1-DE,https://de.surveymonkey.com/survey-thanks/?sm=...,897,9.411233e-08,2302158,IsminiLokkaExperiment1,o30.06.A,11/30/2016,2302522,...,NaN,NaN,NaN,1128.09,564.88,2.99,2.90,364,NaN,o


In [42]:
df2 = (df.groupby(['ParticipantName', 'StudioTestName'], as_index=False).mean()
            .groupby('ParticipantName').mean())

df2

,Blink_Counter,left_avg_pupil_dill,Start_Timestamp,RecordingTimestamp,FixationIndex,SaccadeIndex,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),SaccadicAmplitude,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n
ParticipantName,,,,,,,,,,,,,,,,,
o01.01.A,3037.50000,-2.069561e-06,8.903217e+05,1.137997e+06,1923.849646,3727.393615,10190.170253,590.726148,1769.699688,4.698454,170.010667,176.763975,1011.139703,642.935372,4.012156,3.594565,247674.897522
o02.01.A,2343.50000,-2.628735e-06,6.377481e+05,7.952424e+05,1955.450345,2682.489554,420.475226,611.353925,437.670823,5.178353,192.294595,180.570159,1047.203635,569.357828,4.800838,5.002688,157494.276108
o03.01.A,5455.50000,-4.339637e-06,7.381119e+05,9.366817e+05,2040.530155,3167.136285,371.196991,593.692993,1744.547666,3.912831,174.481227,177.995849,1049.849693,549.000129,5.382580,4.920290,198569.782290
o04.02.A,63.50000,4.963551e-08,1.270209e+06,1.692247e+06,4809.399911,7401.217459,410.780947,594.793215,1643.492317,4.735077,173.526598,183.534568,1076.755042,625.691390,3.133993,3.045703,422038.090888
o05.02.A,1844.56879,-2.646829e-06,6.525738e+05,8.211633e+05,2264.712946,7063.154909,432.844595,555.663473,1255.340345,4.134132,185.469241,178.015898,1000.450549,603.562908,3.556163,3.522000,168589.578579
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
y37.02.A,569.50000,-7.170527e-08,5.207757e+05,6.481532e+05,1864.115157,2467.908383,559.822630,579.261314,1541.655173,5.162107,176.814551,178.349855,1005.182295,562.419290,4.703232,4.585717,127377.500183
y38.01.A,394.50000,4.105912e-06,5.556030e+05,6.933407e+05,1772.701999,2176.515095,484.334791,634.042829,1547.371365,6.662839,179.211979,178.359555,1071.060093,597.882063,3.075016,3.534257,137737.753972
y40.04.A,367.00000,1.041408e-06,5.109857e+05,6.370528e+05,1937.950781,2448.516270,463.121372,596.710381,1527.445996,4.520297,178.907510,175.094247,1033.819966,641.096833,4.159684,4.651945,126067.108722


In [43]:
df['fix_duration'] = (df.groupby(['ParticipantName', 'FixationIndex','RecordingTimestamp'], as_index=False).count()
            .groupby(['ParticipantName','FixationIndex'])['FixationIndex'].transform('count'))


In [44]:
df['fix_duration']

0          86
1          86
2          86
3          86
4          86
         ... 
664288    277
664289    277
664290    277
664291    277
664292    277
Name: fix_duration, Length: 65565830, dtype: int64

In [45]:
df2['fix_duration'] = (df.groupby(['ParticipantName']).mean()
            .groupby(['ParticipantName'])['fix_duration'].mean())
df2

,Blink_Counter,left_avg_pupil_dill,Start_Timestamp,RecordingTimestamp,FixationIndex,SaccadeIndex,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),SaccadicAmplitude,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n,fix_duration
ParticipantName,,,,,,,,,,,,,,,,,,
o01.01.A,3037.50000,-2.069561e-06,8.903217e+05,1.137997e+06,1923.849646,3727.393615,10190.170253,590.726148,1769.699688,4.698454,170.010667,176.763975,1011.139703,642.935372,4.012156,3.594565,247674.897522,148.917205
o02.01.A,2343.50000,-2.628735e-06,6.377481e+05,7.952424e+05,1955.450345,2682.489554,420.475226,611.353925,437.670823,5.178353,192.294595,180.570159,1047.203635,569.357828,4.800838,5.002688,157494.276108,142.892495
o03.01.A,5455.50000,-4.339637e-06,7.381119e+05,9.366817e+05,2040.530155,3167.136285,371.196991,593.692993,1744.547666,3.912831,174.481227,177.995849,1049.849693,549.000129,5.382580,4.920290,198569.782290,141.628203
o04.02.A,63.50000,4.963551e-08,1.270209e+06,1.692247e+06,4809.399911,7401.217459,410.780947,594.793215,1643.492317,4.735077,173.526598,183.534568,1076.755042,625.691390,3.133993,3.045703,422038.090888,160.179908
o05.02.A,1844.56879,-2.646829e-06,6.525738e+05,8.211633e+05,2264.712946,7063.154909,432.844595,555.663473,1255.340345,4.134132,185.469241,178.015898,1000.450549,603.562908,3.556163,3.522000,168589.578579,141.178892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
y37.02.A,569.50000,-7.170527e-08,5.207757e+05,6.481532e+05,1864.115157,2467.908383,559.822630,579.261314,1541.655173,5.162107,176.814551,178.349855,1005.182295,562.419290,4.703232,4.585717,127377.500183,150.864476
y38.01.A,394.50000,4.105912e-06,5.556030e+05,6.933407e+05,1772.701999,2176.515095,484.334791,634.042829,1547.371365,6.662839,179.211979,178.359555,1071.060093,597.882063,3.075016,3.534257,137737.753972,147.848270
y40.04.A,367.00000,1.041408e-06,5.109857e+05,6.370528e+05,1937.950781,2448.516270,463.121372,596.710381,1527.445996,4.520297,178.907510,175.094247,1033.819966,641.096833,4.159684,4.651945,126067.108722,150.971647


In [46]:
df2['Age'] = df2.index.get_level_values('ParticipantName').str[0]

def condition(x):
    if x == "o":
        return 1
    else:
        return 0
df2['Age'] = df2['Age'].apply(condition)

df2

,Blink_Counter,left_avg_pupil_dill,Start_Timestamp,RecordingTimestamp,FixationIndex,SaccadeIndex,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),SaccadicAmplitude,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n,fix_duration,Age
ParticipantName,,,,,,,,,,,,,,,,,,,
o01.01.A,3037.50000,-2.069561e-06,8.903217e+05,1.137997e+06,1923.849646,3727.393615,10190.170253,590.726148,1769.699688,4.698454,170.010667,176.763975,1011.139703,642.935372,4.012156,3.594565,247674.897522,148.917205,1
o02.01.A,2343.50000,-2.628735e-06,6.377481e+05,7.952424e+05,1955.450345,2682.489554,420.475226,611.353925,437.670823,5.178353,192.294595,180.570159,1047.203635,569.357828,4.800838,5.002688,157494.276108,142.892495,1
o03.01.A,5455.50000,-4.339637e-06,7.381119e+05,9.366817e+05,2040.530155,3167.136285,371.196991,593.692993,1744.547666,3.912831,174.481227,177.995849,1049.849693,549.000129,5.382580,4.920290,198569.782290,141.628203,1
o04.02.A,63.50000,4.963551e-08,1.270209e+06,1.692247e+06,4809.399911,7401.217459,410.780947,594.793215,1643.492317,4.735077,173.526598,183.534568,1076.755042,625.691390,3.133993,3.045703,422038.090888,160.179908,1
o05.02.A,1844.56879,-2.646829e-06,6.525738e+05,8.211633e+05,2264.712946,7063.154909,432.844595,555.663473,1255.340345,4.134132,185.469241,178.015898,1000.450549,603.562908,3.556163,3.522000,168589.578579,141.178892,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
y37.02.A,569.50000,-7.170527e-08,5.207757e+05,6.481532e+05,1864.115157,2467.908383,559.822630,579.261314,1541.655173,5.162107,176.814551,178.349855,1005.182295,562.419290,4.703232,4.585717,127377.500183,150.864476,0
y38.01.A,394.50000,4.105912e-06,5.556030e+05,6.933407e+05,1772.701999,2176.515095,484.334791,634.042829,1547.371365,6.662839,179.211979,178.359555,1071.060093,597.882063,3.075016,3.534257,137737.753972,147.848270,0
y40.04.A,367.00000,1.041408e-06,5.109857e+05,6.370528e+05,1937.950781,2448.516270,463.121372,596.710381,1527.445996,4.520297,178.907510,175.094247,1033.819966,641.096833,4.159684,4.651945,126067.108722,150.971647,0


In [47]:
filename = "C:\\Users\\Jeremy_Constantin\\Desktop\\Average_Grade.csv"
avg_grade = pd.read_excel(filename)
avg_grade

,ParticipantName,Average_Grade
0,y01.01.A,0.548548
1,y02.01.A,0.771284
2,y03.01.A,0.580687
3,y04.01.A,0.754847
4,y05.01.A,0.654023
...,...,...
76,o35.05.A,0.445645
77,o36.06.A,0.634114
78,o37.01.A,0.510059
79,o38.02.A,0.497715


In [48]:
resultdf = df2.merge(avg_grade,how="inner",on="ParticipantName")

resultdf

,ParticipantName,Blink_Counter,left_avg_pupil_dill,Start_Timestamp,RecordingTimestamp,FixationIndex,SaccadeIndex,GazeEventDuration,FixationPointX (MCSpx),FixationPointY (MCSpx),...,AbsoluteSaccadicDirection,RelativeSaccadicDirection,StrictAverageGazePointX (ADCSmm),StrictAverageGazePointY (ADCSmm),PupilLeft,PupilRight,Timestamp_n,fix_duration,Age,Average_Grade
0,o01.01.A,3037.50000,-2.069561e-06,8.903217e+05,1.137997e+06,1923.849646,3727.393615,10190.170253,590.726148,1769.699688,...,170.010667,176.763975,1011.139703,642.935372,4.012156,3.594565,247674.897522,148.917205,1,0.604101
1,o02.01.A,2343.50000,-2.628735e-06,6.377481e+05,7.952424e+05,1955.450345,2682.489554,420.475226,611.353925,437.670823,...,192.294595,180.570159,1047.203635,569.357828,4.800838,5.002688,157494.276108,142.892495,1,0.363703
2,o03.01.A,5455.50000,-4.339637e-06,7.381119e+05,9.366817e+05,2040.530155,3167.136285,371.196991,593.692993,1744.547666,...,174.481227,177.995849,1049.849693,549.000129,5.382580,4.920290,198569.782290,141.628203,1,0.582695
3,o04.02.A,63.50000,4.963551e-08,1.270209e+06,1.692247e+06,4809.399911,7401.217459,410.780947,594.793215,1643.492317,...,173.526598,183.534568,1076.755042,625.691390,3.133993,3.045703,422038.090888,160.179908,1,0.631835
4,o05.02.A,1844.56879,-2.646829e-06,6.525738e+05,8.211633e+05,2264.712946,7063.154909,432.844595,555.663473,1255.340345,...,185.469241,178.015898,1000.450549,603.562908,3.556163,3.522000,168589.578579,141.178892,1,0.534927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,y36.02.A,2519.50000,-9.484141e-06,4.899007e+05,5.986756e+05,889.351359,3712.775818,378.382818,676.962277,1194.374588,...,174.316760,177.115445,1181.211972,714.672915,4.427430,4.191877,108774.881412,154.693259,0,0.495673
69,y37.02.A,569.50000,-7.170527e-08,5.207757e+05,6.481532e+05,1864.115157,2467.908383,559.822630,579.261314,1541.655173,...,176.814551,178.349855,1005.182295,562.419290,4.703232,4.585717,127377.500183,150.864476,0,0.568978
70,y38.01.A,394.50000,4.105912e-06,5.556030e+05,6.933407e+05,1772.701999,2176.515095,484.334791,634.042829,1547.371365,...,179.211979,178.359555,1071.060093,597.882063,3.075016,3.534257,137737.753972,147.848270,0,0.627231
71,y40.04.A,367.00000,1.041408e-06,5.109857e+05,6.370528e+05,1937.950781,2448.516270,463.121372,596.710381,1527.445996,...,178.907510,175.094247,1033.819966,641.096833,4.159684,4.651945,126067.108722,150.971647,0,0.733093


In [49]:
resultdf.to_csv(r'C:\Users\Jeremy_Constantin\Desktop\Data_Jeremy.csv')

In [47]:
df.to_csv(r'data_raw_all.csv')

In [48]:
df1_encoding_Path1

,index,ParticipantName,StudioTestName,MediaName,Start_Timestamp,StudioProjectName,RecordingName,RecordingDate,RecordingTimestamp,FixationIndex,...,Inside_B6_AIO_Avg_Gaze,Inside_B6_AIO_Fix_Point,Inside_T3_AIO_Avg_Gaze,Inside_T3_AIO_Fix_Point,Inside_T4_AIO_Avg_Gaze,Inside_T4_AIO_Fix_Point,Inside_B7_AIO_Avg_Gaze,Inside_B7_AIO_Fix_Point,Inside_T5_AIO_Avg_Gaze,Inside_T5_AIO_Fix_Point
0,0,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27246,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27248,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27251,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27255,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27258,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27014,204095,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694860,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27015,204096,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694863,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27016,204097,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694866,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27017,204098,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694870,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
df1_encoding_Path1.to_csv(r'data_encoding_AOI_all.csv')

In [50]:
df1_encoding_Path1

,index,ParticipantName,StudioTestName,MediaName,Start_Timestamp,StudioProjectName,RecordingName,RecordingDate,RecordingTimestamp,FixationIndex,...,Inside_B6_AIO_Avg_Gaze,Inside_B6_AIO_Fix_Point,Inside_T3_AIO_Avg_Gaze,Inside_T3_AIO_Fix_Point,Inside_T4_AIO_Avg_Gaze,Inside_T4_AIO_Fix_Point,Inside_B7_AIO_Avg_Gaze,Inside_B7_AIO_Fix_Point,Inside_T5_AIO_Avg_Gaze,Inside_T5_AIO_Fix_Point
0,0,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27246,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27248,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27251,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27255,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,o02.01.A,01-Part1-DE,P1A.avi,27246,IsminiLokkaExperiment1,o02.01.A,11/3/2016,27258,94.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27014,204095,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694860,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27015,204096,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694863,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27016,204097,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694866,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27017,204098,y04.01.A,01-Part2-EN,P1M.avi,604831,IsminiLokkaExperiment1,y04.01.A,5/24/2016,694870,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
